<a href="https://colab.research.google.com/github/temilola123/Data-Science/blob/main/Spaceship_Titanic_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data Collating**#

In [58]:
import pandas as pd

from google.colab import files


#Upload test file from system
test_uploaded = files.upload()

training_uploaded = files.upload()

Saving test.csv to test (1).csv


Saving train.csv to train (1).csv


In [72]:
import io

#Initialize test_data and train_data to the uploaded files
test_data = pd.read_csv(io.BytesIO(test_uploaded['test (1).csv']))

train_data = pd.read_csv(io.BytesIO(training_uploaded['train (1).csv']))

In [78]:
test_data.head(7)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
5,0027_01,Earth,False,F/7/P,TRAPPIST-1e,31.0,False,0.0,1615.0,263.0,113.0,60.0,Karlen Ricks
6,0029_01,Europa,True,B/2/P,55 Cancri e,21.0,False,0.0,NaN,0.0,0.0,0.0,Aldah Ainserfle


In [71]:
from sklearn.model_selection import train_test_split

y = train_data['Transported']
features = ["CryoSleep", "Destination", "Age", "VIP"]
X_train, X_val, y_train, y_val = train_test_split(train_data, y, train_size = 0.8, random_state = 0 )

In [73]:
categorical_cols = [cname for cname in features if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]

numerical_cols = [cname for cname in features if X_train[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_val = X_val[my_cols].copy()
# Create a copy of test_data with selected columns for prediction
test_data_for_prediction = test_data[my_cols].copy()

#**Preproccessing**#

In [88]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps =(
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
) )

preprocessor = ColumnTransformer(
    transformers=(
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    )
)

model = LogisticRegression(max_iter=10000, random_state=0)

my_pipeline = Pipeline(steps=(
    ('preprocessor', preprocessor),
    ('model', model)
))

my_pipeline.fit(X_train, y_train)

predict = my_pipeline.predict(X_val)

score = mean_absolute_error(y_val, predict)
print('MAE:', score)

MAE: 0.2852213916043703


#**Output**#

In [89]:
# Make predictions on the test data
predict_test = my_pipeline.predict(test_data_for_prediction)

Output = pd.DataFrame({'PassengerId': test_data_original['PassengerId'],
                       'Transported': predict_test})
Output.to_csv('Submission.csv', index=False)


In [90]:
from google.colab import files

files.download('Submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>